In [1]:
from time import perf_counter

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [2]:
model_path = "lexiconium/kogpt-trinity"
revision = "punct_wrapper-related_words-minevalloss"


device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_path, revision=revision, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_path, revision=revision, use_auth_token=True).to(device)
model.eval()

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/715k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/228k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/109 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/847 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.35G [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 1920)
    (wpe): Embedding(1024, 1920)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
     

In [3]:
@torch.no_grad()
def genreate_from_input(input_text: str, max_length: int = 64, top_k: int = 30, top_p: float = 0.95):
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Check generation time
    t = perf_counter()

    output = model.generate(
        input_ids,
        max_length=max_length,
        repetition_penalty=2.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        do_sample=True,
        top_k=top_k,
        top_p=top_p,
        bad_words_ids=[[0], [2], [3], [4], [5], [6], [7], [8]],
    )
    generated_text = tokenizer.decode(output[0])

    spent = perf_counter() - t

    return generated_text, spent

In [5]:
prompt = "스키를 타고 눈 덮인 비탈을 내려오는 남자"
input_text = f"@{prompt}@<usr>\n"

for top_p in [0.95, 0.96, 0.97, 0.98, 0.99]:
    print(f"{top_p=} generated text")
    for n in range(1, 6):
        generated_text, spent = genreate_from_input(input_text, max_length=128, top_p=top_p)
        print(f"{n}:\n{generated_text}")
        # print(f"\ntime spent: {spent:.2f} sec")


top_p=0.95 generated text
1:
@스키를 타고 눈 덮인 비탈을 내려오는 남자@<usr> 
슬레이어스(Slayers)를 다시 보게 되었다
그리고 스키장갑을 벗고
두 손을 꽉 잡았다, 깍지 낀 두 손은
남자가 되고 싶다고 말했었지,
이제는 눈을 감고 내 옆에 앉아,
그렇게 중얼거리며 스릴을 느꼈지
저기 저, 한 남자가 나를 쳐다보고 있지 않아?
내가, 아니, 그이가.
눈물이 뚝뚝 흐르는 눈으로 쳐다보네
한번 더 웃어보이지 않는가.
그러자, 그가 나에게 다가와 속삭였어
너는 이미 남자인걸. 이라고....
뭐? 또 무슨 말을
2:
@스키를 타고 눈 덮인 비탈을 내려오는 남자@<usr> 
우리는 스키장을 한 번만 가본다
눈도 쌓이질 않고
하얀 설원위에 발자욱은 무지
가을철 단풍마냥 빛나기도 전에 사라진다.
그런데 나는 너를 본다,
어둠 속에서 너의 숨결, 네 눈동자 속으로 들어가 본다.
너희들은 어둠속에서 나를 볼까?
내일 다시 이곳을 찾으리라.
나는 너희들의 모습을 사랑한다. 그리고 지금도
오늘 저녁 너에게서 벗어나려 한다. 그러나 결코 벗어날 수 없다.
하지만 내일 아침에는 네가 환한 미소를 지으며 내게 다가올테니
또 언젠가
3:
@스키를 타고 눈 덮인 비탈을 내려오는 남자@<usr> 
나는 스키어를 좋아한다.
그래서 나는 오늘도 눈을 맞으며 내려간다.
눈을 맞을 수록 눈은 더 쌓여만 가고
쌓였던 눈이 녹아 질척해질수록 내 마음도 젖어든다.
그리고
그런 나에게 찾아온 건,
누구도
내려가지 못하게 꽁꽁 얼린 얼음 조각이었다.
그러고 나면, 어느새 해는 지고 난 뒤에야
얼음들이 녹아,내 마음을 적신다. 그 때서야 겨울이 온 것을 알아차린다.
나의 마음은 얼어붙지 않았는데...  나를 차갑게 한 것은 누구였을까?
나를, 나의 감정을. 이 추운 곳에서
4:
@스키를 타고 눈 덮인 비탈을 내려오는 남자@<usr> 
이런, 스키 타는 것 좋아하나?
남자친구가 묻는다.
그건 아닌데, 아니지,
좋아하긴 하는데 위험하지는 않은데 말이야.
뭐 그렇게 생각해야지 안